In [ ]:
!pip install PyQt5

In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PyQt5.QtWidgets import QFileDialog
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import time
import sys
from PyQt5.QtWidgets import (
    QApplication,
    QMainWindow,
    QPushButton,
    QVBoxLayout,
    QWidget,
    QProgressBar,
    QLabel,
    QDialog,
    QVBoxLayout,
)
from PyQt5.QtCore import Qt, QTimer, pyqtProperty
from PyQt5.QtGui import QColor

In [2]:
class VisualizationDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Visualizations")
        self.resize(900, 900)
        layout = QVBoxLayout()
        self.figure_canvases = []
        self.setLayout(layout)

    def add_visualization(self, figure):
        figure_canvas = FigureCanvas(figure)
        self.figure_canvases.append(figure_canvas)
        self.layout().addWidget(figure_canvas)

    def closeEvent(self, event):
        # Clean up the FigureCanvas objects when the dialog is closed.
        for canvas in self.figure_canvases:
            canvas.setParent(None)
            canvas.close()
        super().closeEvent(event)


In [3]:
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.data_manipulator = DataManipulator()
        self.init_ui()

    def init_ui(self):
        self.load_initial_button = QPushButton("Load Initial Data", self)
        self.load_initial_button.clicked.connect(self.handle_load_initial)

        self.load_tx_params_button = QPushButton("Load TxParams Data", self)
        self.load_tx_params_button.clicked.connect(self.handle_load_tx_params)

        self.prepare_button = QPushButton("Prepare Data", self)
        self.prepare_button.clicked.connect(self.handle_prepare)

        self.save_button = QPushButton("Save Prepared Data", self)
        self.save_button.clicked.connect(self.handle_save)

        self.visualize_button = QPushButton("Visualize", self)
        self.visualize_button.clicked.connect(self.handle_visualize)

        self.progress_bar = QProgressBar(self)
        self.progress_bar.setTextVisible(True)
        self.progress_bar.setAlignment(Qt.AlignCenter)
        self.progress_bar.setMaximum(100)
        self.progress_bar.setValue(0)

        self.message_label = QLabel(self)
        self.message_label.setAlignment(Qt.AlignCenter)
        self.message_label.setStyleSheet("color: green; font-weight: bold")

        layout = QVBoxLayout()
        layout.addWidget(self.load_initial_button)
        layout.addWidget(self.load_tx_params_button)
        layout.addWidget(self.prepare_button)
        layout.addWidget(self.save_button)
        layout.addWidget(self.visualize_button)
        layout.addWidget(self.progress_bar)
        layout.addWidget(self.message_label)

        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.setStyleSheet(
            """
            QMainWindow {
                background-color: #f0f0f0;
            }
            QPushButton {
                background-color: #007BFF;
                color: white;
                border: none;
                border-radius: 5px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #0056b3;
            }
            QProgressBar {
                border: 1px solid #ccc;
                border-radius: 5px;
                background-color: #fff;
                color: #000;
                text-align: center;
            }
            QProgressBar::chunk {
                background-color: #007BFF;
            }
            """
        )

        self.setWindowTitle("Data Manipulator")
        self.resize(800, 500)
        self.show()

    def update_progress(self, value):
        self.progress_bar.setValue(value)

    def handle_load_initial(self):
        self.perform_data_task(self.data_manipulator.load_initial_data, 20, "Loading Initial Data...")

    def handle_load_tx_params(self):
        self.perform_data_task(self.data_manipulator.load_tx_params_data, 40, "Loading TxParams Data...")

    def handle_prepare(self):
        self.perform_data_task(self.data_manipulator.prepare_data, 60, "Preparing Data...")

    def handle_save(self):
        self.perform_data_task(self.data_manipulator.save_prepared_data, 80, "Saving Prepared Data...")

    def handle_visualize(self):
        self.perform_data_task(self.data_manipulator.generate_visualization, 100, "Generating Visualization...")
        # Generate the visualizations before opening the dialog
        self.data_manipulator.generate_visualization()

        # Open the visualization dialog
        self.show_visualization_dialog()

    def show_visualization_dialog(self):
        visualization_dialog = VisualizationDialog(self)
        for idx, fig in enumerate(self.data_manipulator.generated_visualizations):
            visualization_dialog.add_visualization(fig)

        visualization_dialog.exec_()
        self.data_manipulator.clear_visualizations()

    def perform_data_task(self, task_function, progress_value, message):
        self.message_label.setText(message)
        self.update_progress(progress_value)
        
        QTimer.singleShot(1000, lambda: self.complete_data_task(task_function, progress_value))

    def complete_data_task(self, task_function, progress_value):
        result = task_function()
        self.update_progress(progress_value)


In [4]:
class DataManipulator:
    def __init__(self):
        self.initial_data = None
        self.prepared_data = None
        self.generated_visualizations = []

    def load_initial_data(self):
        file_name, _ = QFileDialog.getOpenFileName(None, "Open CSV Files", "", "CSV Files (*.csv);;All Files (*)")
        if file_name:
            self.initial_data = pd.read_csv(file_name)
            self.clean_initial_data()


    def clean_initial_data(self):
        
        ngr_values_to_remove = ['NZ02553847', 'SE213515', 'NT05399374', 'NT252675908']
        self.initial_data = self.initial_data[~self.initial_data['NGR'].isin(ngr_values_to_remove)]

        
        if 'EID' in self.initial_data.columns:
            
            self.initial_data['C18A'] = self.initial_data['EID'].apply(lambda x: x if 'C18A' in x else None)
            self.initial_data['C18F'] = self.initial_data['EID'].apply(lambda x: x if 'C18F' in x else None)
            self.initial_data['C188'] = self.initial_data['EID'].apply(lambda x: x if 'C188' in x else None)

        
        self.initial_data[['Site', 'Site Height(m)', 'Aerial height(m)', 'Power(kW)']] = self.initial_data['NGR'].str.extract(r'(?P<Site>.+),(\d+),(\d+),"([0-9,.]+)"')

        
        if self.initial_data['In-Use ERP Total'].dtype == object:
            self.initial_data['In-Use ERP Total'] = pd.to_numeric(self.initial_data['In-Use ERP Total'].str.replace(',', ''), errors='coerce')

        
        if 'Date' in self.initial_data.columns:
            self.initial_data['Date'] = pd.to_datetime(self.initial_data['Date'], format='%d/%m/%Y')

    def generate_visualization(self):
        if self.prepared_data is None or self.prepared_data.empty:
            print("No prepared data available. Preparing the data...")
            self.prepare_data()

        plt.figure(figsize=(8, 5))
        plt.plot(self.prepared_data['Date'], self.prepared_data['In-Use ERP Total'])
        plt.xlabel('Date')
        plt.ylabel('In-Use ERP Total')
        plt.title('DAB Multiplexes: C18A, C18F, C188')
        plt.tight_layout()
        self.generated_visualizations.append(plt.gcf())

        erp_stats = self.calculate_erp_stats()
        if not erp_stats.empty:
            plt.figure(figsize=(8, 5))
            sns.barplot(x='Metric', y='Value', data=erp_stats)
            plt.xlabel('Metric')
            plt.ylabel('ERP Value')
            plt.title('Mean, Mode, and Median for In-Use ERP Total')
            plt.tight_layout()
            self.generated_visualizations.append(plt.gcf())

        columns_to_analyze = ['Freq.', 'Block', 'In-Use ERP Total']
        data_for_pairplot = self.prepared_data[columns_to_analyze]
        if not data_for_pairplot.empty:
            sns.pairplot(data_for_pairplot)
            plt.suptitle('Correlation Analysis', y=1.02)
            plt.tight_layout()
            self.generated_visualizations.append(plt.gcf())

    def clear_visualizations(self):
        for fig in self.generated_visualizations:
            plt.close(fig)
        self.generated_visualizations = []

    def calculate_erp_stats(self):
        if self.prepared_data is None or self.prepared_data.empty:
            print("No prepared data available. Preparing the data...")
            self.prepare_data()

        if 'In-Use ERP Total' not in self.prepared_data.columns:
            print("Unable to calculate ERP statistics. 'In-Use ERP Total' column not found in the data.")
            return pd.DataFrame({'Metric': [], 'Value': []})

        erp_mean = self.prepared_data['In-Use ERP Total'].mean()

        
        if len(self.prepared_data['In-Use ERP Total']) > 0:
            erp_mode = self.prepared_data['In-Use ERP Total'].mode().values[0]
        else:
            erp_mode = None

        erp_median = self.prepared_data['In-Use ERP Total'].median()

        return pd.DataFrame({'Metric': ['Mean', 'Mode', 'Median'],
                             'Value': [erp_mean, erp_mode, erp_median]})
    
    def prepare_data(self):

        if 'Date' not in self.initial_data.columns:
            print("No 'Date' column found in the data. Please load the data with 'Date' column.")
            self.prepared_data = self.initial_data.copy()
            return

        self.clean_initial_data()
        
        filtered_data = self.initial_data[
            (self.initial_data['In-Use ERP Total'] >= self.initial_data['In-Use ERP Total'].min()) &
            (self.initial_data['In-Use Ae Ht'] > 100) &
            (self.initial_data['Date'].dt.year >= 2001)
        ]

        self.prepared_data = filtered_data.copy()

    def save_prepared_data(self):
        file_name, _ = QFileDialog.getSaveFileName(None, "Save Data", "", "JSON Files (*.json);;All Files (*)")
        if file_name:
            self.prepared_data.to_json(file_name, orient='records')

    def load_tx_params_data(self):
        file_name, _ = QFileDialog.getOpenFileName(None, "Open CSV Files", "", "CSV Files (*.csv);;All Files (*)")
        if file_name:
            tx_params_data = pd.read_csv(file_name, encoding='latin1', parse_dates=['Date'], dayfirst=True)
            self.merge_datasets(tx_params_data)

    def merge_datasets(self, tx_params_data):
        if self.initial_data is None:
            raise ValueError("No initial data loaded. Please load the initial data first.")

        
        self.initial_data = self.initial_data.merge(tx_params_data, on='id', how='left')


In [5]:
class ColorButton(QPushButton):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self._color = QColor("#007BFF")
        self._base_color = self._color

    def setColor(self, color):
        if color != self._color:
            self._color = color
            self.style().polish(self)

    def getColor(self):
        return self._color

    color = pyqtProperty(QColor, getColor, setColor)


In [6]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = MainWindow()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\muham\anaconda_3\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
